# Question Answering Model 
## no trainer

- dataset
- torch
- transformers
- transformers[torch]
- evaluate

import packages

In [1]:
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    DefaultDataCollator,
    default_data_collator,
    get_scheduler,
    AutoModelForQuestionAnswering,
    TrainingArguments,
    Trainer
)
from torch.utils.data import DataLoader
from torch.optim import AdamW
import torch

import evaluate
import collections
from tqdm.auto import tqdm
import numpy as np

import os
import re
import datetime

/mount/arbeitsdaten31/studenten1/linku/.venv/lib64/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Set cache directory.

In [2]:
model_dir = '/mount/arbeitsdaten31/studenten1/linku/models'
CACHE_DIR='/mount/arbeitsdaten31/studenten1/linku/cache'
%set_env TRANSFORMERS_CACHE $CACHE_DIR
%set_env HF_MODULES_CACHE $CACHE_DIR
%set_env HF_DATASETS_CACHE $CACHE_DIR

env: TRANSFORMERS_CACHE=/mount/arbeitsdaten31/studenten1/linku/cache
env: HF_MODULES_CACHE=/mount/arbeitsdaten31/studenten1/linku/cache
env: HF_DATASETS_CACHE=/mount/arbeitsdaten31/studenten1/linku/cache


### arguments.py

args_input.

In [3]:
args_input_ALstrategy = 'MarginSampling'
args_input_initseed = 100 # 1000
args_input_quota = 100 # 1000
args_input_batch = 35 # 128
args_input_dataset_name = 'SQuAD'
args_input_iteration = 1
args_input_model_batch = 8 # already add in arguments.py

### load dataset

In [4]:
squad = load_dataset(args_input_dataset_name.lower())
# squad["train"] = squad["train"].shuffle(42).select(range(2000))
squad["train"] = squad["train"].select(range(3000))
squad["validation"] = squad["validation"].select(range(1000))

Found cached dataset squad (/home/users1/linku/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453)
100%|██████████| 2/2 [00:00<00:00, 77.96it/s]


Next we will preprocess the dataset (training and evaluation data).

In [5]:
def preprocess_training_features(examples):
    # keep ["offset_mapping"], for compute_metrics()
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs["offset_mapping"]
    sample_map = inputs.pop("overflow_to_sample_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []
    example_ids = []

    for i, offset in enumerate(offset_mapping):
        sample_idx = sample_map[i]
        answer = answers[sample_idx]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)
        
        example_ids.append(examples["id"][sample_idx]) # newly added for used in unlabel data predict

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label is (0, 0)
        if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["example_id"] = example_ids
    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [6]:
def preprocess_training_examples(examples):
    # no ['offset_mapping'], for .train() and .eval()
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    sample_map = inputs.pop("overflow_to_sample_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []
    example_ids = []

    for i, offset in enumerate(offset_mapping):
        sample_idx = sample_map[i]
        answer = answers[sample_idx]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)
        
        example_ids.append(examples["id"][sample_idx]) # newly added for used in unlabel data predict

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label is (0, 0)
        if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [7]:
def preprocess_validation_examples(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_map = inputs.pop("overflow_to_sample_mapping")
    example_ids = []

    for i in range(len(inputs["input_ids"])):
        sample_idx = sample_map[i]
        example_ids.append(examples["id"][sample_idx])

        sequence_ids = inputs.sequence_ids(i)
        offset = inputs["offset_mapping"][i]
        inputs["offset_mapping"][i] = [
            o if sequence_ids[k] == 1 else None for k, o in enumerate(offset)
        ]

    inputs["example_id"] = example_ids
    return inputs

In [8]:
max_length = 384
stride = 128

# load tokenizer for dataset preprocessing
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
# preprocess data
train_dataset = squad["train"].map(
    preprocess_training_examples,
    batched=True,
    remove_columns=squad["train"].column_names,
)
train_features = squad["train"].map(
    preprocess_training_features,
    batched=True,
    remove_columns=squad["train"].column_names,
)
val_dataset = squad["validation"].map(
    preprocess_validation_examples,
    batched=True,
    remove_columns=squad["validation"].column_names,
)
val_features = squad["validation"].map(
    preprocess_validation_examples,
    batched=True,
    remove_columns=squad["validation"].column_names,
)

Loading cached processed dataset at /home/users1/linku/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453/cache-d10ddabde563b597.arrow
Loading cached processed dataset at /home/users1/linku/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453/cache-773e51dc4558cf67.arrow
Loading cached processed dataset at /home/users1/linku/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453/cache-0f087a92e258c27b.arrow
Loading cached processed dataset at /home/users1/linku/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453/cache-0f087a92e258c27b.arrow


In [9]:
train_dataset.set_format("torch")
train_features.set_format("torch")
val_dataset = val_dataset.remove_columns(["offset_mapping"])
val_dataset.set_format("torch")
val_features.set_format("torch")

## Evaluation

In [10]:
metric = evaluate.load("squad")

In [11]:
def compute_metrics(start_logits, end_logits, features, examples):
    
    example_to_features = collections.defaultdict(list)
    max_answer_length = 30
    n_best = 20
    for idx, feature in enumerate(features):
        example_to_features[feature["example_id"]].append(idx)

    predicted_answers = []
    for example in tqdm(examples):
        example_id = example["id"]
        context = example["context"]
        answers = []

        # Loop through all features associated with that example
        for feature_index in example_to_features[example_id]:
            start_logit = start_logits[feature_index]
            end_logit = end_logits[feature_index]
            offsets = features[feature_index]["offset_mapping"]

            start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
            end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Skip answers that are not fully in the context
                    if offsets[start_index] is None or offsets[end_index] is None:
                        continue
                    # Skip answers with a length that is either < 0 or > max_answer_length
                    if (
                        end_index < start_index
                        or end_index - start_index + 1 > max_answer_length
                    ):
                        continue

                    answer = {
                        "text": context[offsets[start_index][0] : offsets[end_index][1]],
                        "logit_score": start_logit[start_index] + end_logit[end_index],
                    }
                    answers.append(answer)

        # Select the answer with the best score
        if len(answers) > 0:
            best_answer = max(answers, key=lambda x: x["logit_score"])
            predicted_answers.append(
                {"id": example_id, "prediction_text": best_answer["text"]}
            )
        else:
            predicted_answers.append({"id": example_id, "prediction_text": ""})

    theoretical_answers = [{"id": ex["id"], "answers": ex["answers"]} for ex in examples]
    return metric.compute(predictions=predicted_answers, references=theoretical_answers)

In [12]:
def get_pred(model, eval_dataloader, device, features, examples):
    model.eval()
    start_logits = []
    end_logits = []
    # accelerator.print("Evaluation!")
    for batch in tqdm(eval_dataloader):
        batch = {key: value.to(device) for key, value in batch.items()}
        with torch.no_grad():
            outputs = model(**batch)

        start_logits.append(outputs.start_logits.cpu().numpy())
        end_logits.append(outputs.end_logits.cpu().numpy())

    start_logits = np.concatenate(start_logits)
    end_logits = np.concatenate(end_logits)
    start_logits = start_logits[: len(features)]
    end_logits = end_logits[: len(features)]

    return compute_metrics(start_logits, end_logits, features, examples)

In [13]:
def get_prob(model, eval_dataloader, device, features, examples):
    model.eval()
    start_logits = []
    end_logits = []
    # accelerator.print("Evaluation!")
    for batch in tqdm(eval_dataloader):
        batch = {key: value.to(device) for key, value in batch.items()}
        with torch.no_grad():
            outputs = model(**batch)

        start_logits.append(outputs.start_logits.cpu().numpy())
        end_logits.append(outputs.end_logits.cpu().numpy())

    start_logits = np.concatenate(start_logits)
    end_logits = np.concatenate(end_logits)
    start_logits = start_logits[: len(features)]
    end_logits = end_logits[: len(features)]

    prob_dict = {}
    example_to_features = collections.defaultdict(list)
    max_answer_length = 30
    n_best = 20 # TODO: if set n_best as 5, will it effect the time??
    
    for idx, feature in enumerate(features):
        example_to_features[feature["example_id"]].append(idx)

    for example in tqdm(examples):
        example_id = example["id"]
        # context = example["context"]
        answers = []

        # Loop through all features associated with that example
        for feature_index in example_to_features[example_id]:
            start_logit = start_logits[feature_index]
            end_logit = end_logits[feature_index]
            offsets = features[feature_index]["offset_mapping"]

            start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
            end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Skip answers that are not fully in the context
                    if offsets[start_index] is None or offsets[end_index] is None:
                        continue
                    # Skip answers with a length that is either < 0 or > max_answer_length
                    if (
                        end_index < start_index
                        or end_index - start_index + 1 > max_answer_length
                    ):
                        continue

                    answers.append(start_logit[start_index] + end_logit[end_index])
        
        if len(answers) > 1:
            if example_to_features[example_id][0] not in prob_dict:
                prob_dict[example_to_features[example_id][0]] = softmax(answers)
            else:
                prob_dict[example_to_features[example_id][0]] += softmax(answers)
        elif example_to_features[example_id] != []:
            if example_to_features[example_id][0] not in prob_dict:
                prob_dict[example_to_features[example_id][0]] = np.array([0])
    
    return prob_dict
# move to evaluation.py

In [14]:
def get_prob_dropout(model, eval_dataloader, device, features, examples, n_drop=10):
    model.train()
    
    prob_dict = {}
    for_check = []
    
    for i in range(n_drop):
        
        start_logits = []
        end_logits = []
        # accelerator.print("Evaluation!")
        for batch in tqdm(eval_dataloader):
            batch = {key: value.to(device) for key, value in batch.items()}
            with torch.no_grad():
                outputs = model(**batch)

            start_logits.append(outputs.start_logits.cpu().numpy())
            end_logits.append(outputs.end_logits.cpu().numpy())

        start_logits = np.concatenate(start_logits)
        end_logits = np.concatenate(end_logits)
        start_logits = start_logits[: len(features)]
        end_logits = end_logits[: len(features)]

        example_to_features = collections.defaultdict(list)
        max_answer_length = 30
        n_best = 20
            
        for idx, feature in enumerate(features):
            example_to_features[feature["example_id"]].append(idx)

        n = 0
        for example in tqdm(examples):
            example_id = example["id"]
            # context = example["context"]
            answers = []

            # Loop through all features associated with that example
            for feature_index in example_to_features[example_id]:
                start_logit = start_logits[feature_index]
                end_logit = end_logits[feature_index]
                offsets = features[feature_index]["offset_mapping"]

                start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
                end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
                for start_index in start_indexes:
                    for end_index in end_indexes:
                        # Skip answers that are not fully in the context
                        if offsets[start_index] is None or offsets[end_index] is None:
                            continue
                        # Skip answers with a length that is either < 0 or > max_answer_length
                        if (
                            end_index < start_index
                            or end_index - start_index + 1 > max_answer_length
                        ):
                            continue

                        answers.append(start_logit[start_index] + end_logit[end_index])

            
            if 1 < len(answers) < 150:
                zero_list = [0] * (150 - len(answers))
                answers.extend(zero_list)
            else:
                answers[:150]

            if len(answers) > 1:
                if example_to_features[example_id][0] not in prob_dict:
                    prob_dict[example_to_features[example_id][0]] = softmax(answers)
                else:
                    prob_dict[example_to_features[example_id][0]] += softmax(answers)
            elif example_to_features[example_id] != []:
                if example_to_features[example_id][0] not in prob_dict:
                    prob_dict[example_to_features[example_id][0]] = np.array([0])
            # if n == 0 and len(softmax(answers)) > 1:
            #     for_check.append(answers[:5])
            #     n += 1      

    for key in prob_dict.keys():
        prob_dict[key] /= n_drop
    # return prob_dict, for_check
    return prob_dict
# move to evaluation.py

## utils.py

In [15]:
def get_unlabel_data(n_pool, labeled_idxs, train_dataset, train_feature):
    unlabeled_idxs = np.arange(n_pool)[~labeled_idxs]
    unlabeled_data = train_dataset.select(indices=unlabeled_idxs)
    return unlabeled_idxs, unlabeled_data

# move to utils.py

In [16]:
def softmax(x): 
    """Compute softmax values for each sets of scores in x."""
    return np.exp(x) / np.sum(np.exp(x), axis=0)
# move to utils.py

## Query

In [17]:
def random_sampling_query(labeled_idxs, n):
    return np.random.choice(np.where(labeled_idxs==0)[0], n, replace=False)

In [49]:
# import DataLoader
# import default_data_collator
import itertools
def margin_sampling_query(n_pool, labeled_idxs, train_dataset, train_features, examples, model, device, n):
    # deepAL+: unlabeled_idxs, unlabeled_data = self.dataset.get_unlabeled_data()
    unlabeled_idxs, unlabeled_data = get_unlabel_data(n_pool, labeled_idxs, train_dataset)
    unlabeled_features = train_features.select(unlabeled_idxs)
    
    unlabeled_dataloader = DataLoader(
		unlabeled_data,
		shuffle=True,
		collate_fn=default_data_collator,
		batch_size=8,
	)

    # deepAL+: probs = self.predict_prob(unlabeled_data)
    prob_dict = get_prob(model, unlabeled_dataloader, device, unlabeled_features, examples)
    
    # deepAL+: probs_sorted, _ = probs.sort(descending=True)
    # deepAL+: uncertainties = probs_sorted[:, 0] - probs_sorted[:,1]
    uncertainties_dict = {}
    for idx, probs in prob_dict.items():
        if len(probs) > 1: # if prob_dict['probs'] is not 0
            sort_probs = np.sort(probs)[::-1] # This method returns a copy of the array, leaving the original array unchanged.
            uncertainties = sort_probs[0] - sort_probs[1]
            uncertainties_dict[idx] = uncertainties
        elif idx:
            uncertainties_dict[idx] = np.array([0])

    # deepAL+: return unlabeled_idxs[uncertainties.sort()[1][:n]] 
    sorted_uncertainties_dict = sorted(uncertainties_dict.items(), key=lambda x: x[1], reverse=True)
    
    return unlabeled_idxs[[idx for (idx, uncertainties) in sorted_uncertainties_list[:n]]]

In [19]:
def least_confidence_query(n_pool, labeled_idxs, train_dataset, trainer_qs, example, n):
    # deepAL+: unlabeled_idxs, unlabeled_data = self.dataset.get_unlabeled_data()
    unlabeled_idxs, unlabeled_data = get_unlabel_data(n_pool, labeled_idxs, train_dataset)
    
    # deepAL+: probs = self.predict_prob(unlabeled_data)
    prob_list_dict = get_prob(trainer_qs, unlabeled_data, example)
    
    # deepAL+: uncertainties = probs.max(1)[0]
    confidence_list_dict = []
    for d in prob_list_dict:
        if len(d['probs']) > 1: # if prob_dict['probs'] is not 0
            confidence = max(d['probs'])
            confidence_list_dict.append(
                {'idx': d['idx'], 
                    'confidence': confidence}
                    )
        elif d['idx']:
            confidence_list_dict.append(
                {'idx': d['idx'], 
                    'confidence': np.array([0])}
                    )
    # deepAL+: return unlabeled_idxs[uncertainties.sort()[1][:n]]
    sorted_confidence_dict = sorted(confidence_list_dict, key=lambda d: d['confidence'])   
    return unlabeled_idxs[[confidence_dict['idx'][0] for confidence_dict in sorted_confidence_dict[:n]]]

In [20]:
def var_ratio_query(n_pool, labeled_idxs, train_dataset, trainer_qs, example, n):
    # deepAL+: unlabeled_idxs, unlabeled_data = self.dataset.get_unlabeled_data()
    unlabeled_idxs, unlabeled_data = get_unlabel_data(n_pool, labeled_idxs, train_dataset)
    
    # deepAL+: probs = self.predict_prob(unlabeled_data)
    prob_list_dict = get_prob(trainer_qs, unlabeled_data, example)
    
    # deepAL+: preds = torch.max(probs, 1)[0]
    # deepAL+: uncertainties = 1.0 - preds
    confidence_list_dict = []
    for d in prob_list_dict:
        if len(d['probs']) > 1: # if prob_dict['probs'] is not 0
            confidence = max(d['probs'])
            confidence_list_dict.append(
                {'idx': d['idx'], 
                    'confidence': 1 - confidence}
                    )
        elif d['idx']:
            confidence_list_dict.append(
                {'idx': d['idx'], 
                    'confidence': np.array([0])}
                    )

    # deepAL+: return unlabeled_idxs[uncertainties.sort(descending=True)[1][:n]]
    sorted_confidence_dict = sorted(confidence_list_dict, key=lambda d: d['confidence'], reverse=True)
    return unlabeled_idxs[[confidence_dict['idx'][0] for confidence_dict in sorted_confidence_dict[:n]]]
# comment for the same query as LC

In [21]:
def entropy_query(n_pool, labeled_idxs, train_dataset, trainer_qs, example, n):
    # deepAL+: unlabeled_idxs, unlabeled_data = self.dataset.get_unlabeled_data()
    unlabeled_idxs, unlabeled_data = get_unlabel_data(n_pool, labeled_idxs, train_dataset)
    
    # deepAL+: probs = self.predict_prob(unlabeled_data)
    prob_list_dict = get_prob(trainer_qs, unlabeled_data, example)
    
    # deepAL+: log_probs = torch.log(probs)
    # deepAL+: uncertainties = (probs*log_probs).sum(1)
    entropy_list_dict = []
    for d in prob_list_dict:
        if len(d['probs']) > 1: # if prob_dict['probs'] is not 0
            log_probs = np.log(d['probs'])
            entropy_list_dict.append(
                {'idx': d['idx'], 
                'entropy': (d['probs']*log_probs).sum()}
                )
        elif d['idx']:
            entropy_list_dict.append(
                {'idx': d['idx'], 
                'entropy': np.array([0])}
                )
    # deepAL+: return unlabeled_idxs[uncertainties.sort()[1][:n]] # use smallest Entropy
    sorted_entropy_dict = sorted(entropy_list_dict, key=lambda d: d['entropy'], reverse=True) # use largest Entropy, different from deepAL+
    return unlabeled_idxs[[entropy_dict['idx'][0] for entropy_dict in sorted_entropy_dict[:n]]]

# main.py

### parameters

In [22]:
NUM_QUERY = args_input_batch
NUM_INIT_LB = args_input_initseed
NUM_ROUND = int(args_input_quota / args_input_batch)
DATA_NAME = args_input_dataset_name
STRATEGY_NAME = args_input_ALstrategy

### seed and device

In [23]:
SEED = 4666
# os.environ['TORCH_HOME']='./basicmodel'
os.environ["CUDA_VISIBLE_DEVICES"] = str(3)

# fix random seed
np.random.seed(SEED)
torch.manual_seed(SEED)
# torch.backends.cudnn.enabled  = True
# torch.backends.cudnn.benchmark= True

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### start experiment

In [24]:
iteration = args_input_iteration
model_batch = args_input_model_batch
num_train_epochs = 3

all_acc = []
acq_time = []

# Change "fp16_training" to True to support automatic mixed precision training (fp16)	
fp16_training = False

if fp16_training:
    !pip install accelerate==0.2.0
    from accelerate import Accelerator
    accelerator = Accelerator(fp16=True)
    device = accelerator.device

In [25]:
def model_train(num_training_steps, num_train_epochs, train_dataloader, device, model, optimizer, lr_scheduler):
	if fp16_training:
		model, optimizer, train_dataloader = accelerator.prepare(model, optimizer, train_dataloader)
	
	progress_bar = tqdm(range(num_training_steps))
	for epoch in range(num_train_epochs):
		# Training
		model.train()
		for step, batch in enumerate(train_dataloader):
			batch = {key: value.to(device) for key, value in batch.items()}
			outputs = model(**batch)
			loss = outputs.loss

			if fp16_training:
				accelerator.backward(loss)
			else:
				loss.backward()

			optimizer.step()
			lr_scheduler.step()
			optimizer.zero_grad()
			progress_bar.update(1)

In [26]:
# repeate # iteration trials
while (iteration > 0): 
	iteration = iteration - 1

	## data, network, strategy
	model = AutoModelForQuestionAnswering.from_pretrained("bert-base-uncased").to(device)
	optimizer = AdamW(model.parameters(), lr=1e-4)

	start = datetime.datetime.now()

	## generate initial labeled pool
	n_pool = len(train_dataset)
	labeled_idxs = np.zeros(n_pool, dtype=bool)

	tmp_idxs = np.arange(n_pool)
	np.random.shuffle(tmp_idxs)
	labeled_idxs[tmp_idxs[:args_input_initseed]] = True

	run_0_labeled_idxs = np.arange(n_pool)[labeled_idxs] 
	
	## record acc performance 
	acc = np.zeros(NUM_ROUND + 1) # build 3 runs + run_0 # origin 10 runs + run_0
	
	## load the selected train data to DataLoader
	train_dataloader = DataLoader(
		train_dataset.select(indices=run_0_labeled_idxs),
		shuffle=True,
		collate_fn=default_data_collator,
		batch_size=8,
	)

	eval_dataloader = DataLoader(
		val_dataset, 
		collate_fn=default_data_collator, 
		batch_size=8
	)

	num_update_steps_per_epoch = len(train_dataloader)
	num_training_steps = num_train_epochs * num_update_steps_per_epoch

	lr_scheduler = get_scheduler(
		"linear",
		optimizer=optimizer,
		num_warmup_steps=0,
		num_training_steps=num_training_steps,
	)


	## print info
	print(DATA_NAME)
	# print('RANDOM SEED {}'.format(SEED))
	print(STRATEGY_NAME) # print(type(strategy).__name__)
	
	## round 0 accuracy 
	model_train(num_training_steps, num_train_epochs, train_dataloader, device, model, optimizer, lr_scheduler)

	acc[0] = get_pred(model, eval_dataloader, device, val_features, squad['validation'])['f1']

	print('Round 0\ntesting accuracy {}'.format(acc[0]))
	print('\n')
	
	## round 1 to rd
	for rd in range(1, NUM_ROUND+1):
		print('Round {}'.format(rd))

		## query
		if STRATEGY_NAME == 'RandomSampling':
			q_idxs = random_sampling_query(labeled_idxs, NUM_QUERY)
		elif STRATEGY_NAME == 'MarginSampling':
			q_idxs = margin_sampling_query(n_pool, labeled_idxs, train_dataset, trainer_qs, squad['train'], NUM_QUERY)
		elif STRATEGY_NAME == 'LeastConfidence':
			q_idxs = least_confidence_query(n_pool, labeled_idxs, train_dataset, trainer_qs, squad['train'], NUM_QUERY)
		elif STRATEGY_NAME == 'EntropySampling':
			q_idxs = entropy_query(n_pool, labeled_idxs, train_dataset, trainer_qs, squad['train'], NUM_QUERY)
		elif STRATEGY_NAME == 'MarginSamplingDropout':
			q_idxs = margin_sampling_dropout_query(n_pool, labeled_idxs, train_dataset, trainer_qs, squad['train'], NUM_QUERY)
		elif STRATEGY_NAME == 'LeastConfidenceDropout':
			q_idxs = least_confidence_dropout_query(n_pool, labeled_idxs, train_dataset, trainer_qs, squad['train'], NUM_QUERY)
		elif STRATEGY_NAME == 'EntropySamplingDropout':
			q_idxs = entropy_dropout_query(n_pool, labeled_idxs, train_dataset, trainer_qs, squad['train'], NUM_QUERY)
		elif STRATEGY_NAME == 'VarRatio':
			q_idxs = var_ratio_query(n_pool, labeled_idxs, train_dataset, trainer_qs, squad['train'], NUM_QUERY)
		
		## update
		labeled_idxs[q_idxs] = True
		run_rd_labeled_idxs = np.arange(n_pool)[labeled_idxs]
		
		## load the selected train data to DataLoader
		train_dataloader_rd = DataLoader(
			train_dataset.select(indices=run_rd_labeled_idxs),
			shuffle=True,
			collate_fn=default_data_collator,
			batch_size=8,
		)

		## train
		model_train(num_training_steps, num_train_epochs, train_dataloader, device, model, optimizer, lr_scheduler)

		## round rd accuracy
		acc[rd] = get_pred(model, eval_dataloader, device, val_features, squad['validation'])['f1']
		print('testing accuracy {}'.format(acc[rd]))
		print('\n')

		torch.cuda.empty_cache()
	
	## print results
	print('SEED {}'.format(SEED))
	print(STRATEGY_NAME)
	print(acc)
	all_acc.append(acc)
	
	## record acq time
	timestamp = re.sub('\.[0-9]*','_',str(datetime.datetime.now())).replace(" ", "_").replace("-", "").replace(":","")
	model_saved_dir = model_dir + '/' + timestamp + '/train_bert_squad_' + str(rd)
	end = datetime.datetime.now()
	acq_time.append(round(float((end-start).seconds), 3))
	torch.save(model, model_saved_dir)

# cal mean & standard deviation
acc_m = []
file_name_res_tot = DATA_NAME+ '_'  + STRATEGY_NAME + '_' + str(NUM_QUERY) + '_' + str(NUM_INIT_LB) +  '_' + str(args_input.quota) + '_normal_res_tot.txt'
file_res_tot =  open(os.path.join(os.path.abspath('') + '/results', '%s' % file_name_res_tot),'w')

file_res_tot.writelines('dataset: {}'.format(DATA_NAME) + '\n')
file_res_tot.writelines('AL strategy: {}'.format(STRATEGY_NAME) + '\n')
file_res_tot.writelines('number of labeled pool: {}'.format(NUM_INIT_LB) + '\n')
file_res_tot.writelines('number of unlabeled pool: {}'.format(len(train_dataset) - NUM_INIT_LB) + '\n')
file_res_tot.writelines('number of testing pool: {}'.format(len(val_dataset)) + '\n')
file_res_tot.writelines('batch size: {}'.format(NUM_QUERY) + '\n')
file_res_tot.writelines('quota: {}'.format(NUM_ROUND*NUM_QUERY)+ '\n')
file_res_tot.writelines('time of repeat experiments: {}'.format(args_input.iteration)+ '\n')

# result
for i in range(len(all_acc)):
	acc_m.append(get_aubc(args_input.quota, NUM_QUERY, all_acc[i]))
	print(str(i)+': '+str(acc_m[i]))
	file_res_tot.writelines(str(i)+': '+str(acc_m[i])+'\n')
mean_acc, stddev_acc = get_mean_stddev(acc_m)
mean_time, stddev_time = get_mean_stddev(acq_time)

print('mean AUBC(acc): '+str(mean_acc)+'. std dev AUBC(acc): '+str(stddev_acc))
print('mean time: '+str(mean_time)+'. std dev time: '+str(stddev_time))

file_res_tot.writelines('mean acc: '+str(mean_acc)+'. std dev acc: '+str(stddev_acc)+'\n')
file_res_tot.writelines('mean time: '+str(mean_time)+'. std dev acc: '+str(stddev_time)+'\n')

# save result
file_name_res = DATA_NAME+ '_'  + STRATEGY_NAME + '_' + str(NUM_QUERY) + '_' + str(NUM_INIT_LB) +  '_' + str(args_input.quota) + '_normal_res.txt'
file_res =  open(os.path.join(os.path.abspath('') + '/results', '%s' % file_name_res),'w')


file_res.writelines('dataset: {}'.format(DATA_NAME) + '\n')
file_res.writelines('AL strategy: {}'.format(STRATEGY_NAME) + '\n')
file_res.writelines('number of labeled pool: {}'.format(NUM_INIT_LB) + '\n')
# file_res.writelines('number of unlabeled pool: {}'.format(dataset.n_pool - NUM_INIT_LB) + '\n')
file_res.writelines('number of unlabeled pool: {}'.format(len(train_dataset) - NUM_INIT_LB) + '\n')
# file_res.writelines('number of testing pool: {}'.format(dataset.n_test) + '\n')
file_res.writelines('number of testing pool: {}'.format(len(val_dataset)) + '\n')
file_res.writelines('batch size: {}'.format(NUM_QUERY) + '\n')
file_res.writelines('quota: {}'.format(NUM_ROUND*NUM_QUERY)+ '\n')
file_res.writelines('time of repeat experiments: {}'.format(args_input.iteration)+ '\n')
avg_acc = np.mean(np.array(all_acc),axis=0)
for i in range(len(avg_acc)):
	tmp = 'Size of training set is ' + str(NUM_INIT_LB + i*NUM_QUERY) + ', ' + 'accuracy is ' + str(round(avg_acc[i],4)) + '.' + '\n'
	file_res.writelines(tmp)

file_res.close()
file_res_tot.close()


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForQuestionAnswering: ['cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_out

SQuAD
MarginSampling


100%|██████████| 1000/1000 [00:09<00:00, 102.49it/s]


In [28]:
acc

array([16.34348008,  0.        ,  0.        ])

## query workspace

In [30]:
# get unlable data
unlabeled_idxs = np.arange(n_pool)[~labeled_idxs]
unlabeled_data = train_dataset.select(indices=unlabeled_idxs)
len(unlabeled_idxs)

2974

### test: query 5 data from 20 unlabeled_data

In [42]:
# smaller data
unlabeled_idxs_20 = unlabeled_idxs[20:40]
unlabeled_data_20 = train_dataset.select(unlabeled_idxs_20)
unlabeled_feature_20 = train_features.select(unlabeled_idxs_20)
len(unlabeled_data_20)

20

In [43]:
unlabeled_idxs_20

array([20, 21, 22, 23, 24, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37,
       38, 39, 40])

In [44]:
unlabeled_dataloader = DataLoader(
		unlabeled_data_20,
		shuffle=True,
		collate_fn=default_data_collator,
		batch_size=8,
	)

In [45]:
probs_list_dict_20 = get_prob(model, unlabeled_dataloader, device, unlabeled_feature_20, squad['train'])
# len(probs_list_dict_20)

100%|██████████| 3000/3000 [00:00<00:00, 3829.59it/s]


In [38]:
probs_list_dict_20

{0: array([0.07559638, 0.05358176, 0.04246078, 0.03847675, 0.02871645,
        0.02713737, 0.04639477, 0.03605317, 0.03241921, 0.0239022 ,
        0.02320026, 0.01954072, 0.01561525, 0.01165528, 0.01564988,
        0.01211285, 0.01168001, 0.01103775, 0.05603519, 0.02389271,
        0.01693485, 0.01342   , 0.01216083, 0.00907602, 0.00857694,
        0.02903107, 0.01237849, 0.00877372, 0.00695272, 0.00630036,
        0.00470216, 0.00410473, 0.00404521, 0.02572316, 0.01096804,
        0.00777401, 0.00445864, 0.00363702, 0.00358428, 0.00549511,
        0.00497951, 0.00385409, 0.00371637, 0.00351201, 0.01052027,
        0.00541996, 0.00526079, 0.00470922, 0.01899265, 0.00809823,
        0.00573992, 0.00329203, 0.00289546, 0.00268539, 0.00264645,
        0.01777625, 0.00388845, 0.00308119, 0.00271002, 0.0025134 ,
        0.00247696, 0.00647306, 0.00333487, 0.00323693, 0.00289756,
        0.00447262, 0.00251217, 0.00227646, 0.001699  , 0.00160557,
        0.00217191, 0.00168104, 0.00153183, 0

In [48]:
uncertainties_dict = {}
for idx, probs in probs_list_dict_20.items():
    if len(probs) > 1: # if prob_dict['probs'] is not 0
        sort_probs = np.sort(probs)[::-1] # This method returns a copy of the array, leaving the original array unchanged.
        uncertainties = sort_probs[0] - sort_probs[1]
        uncertainties_dict[idx] = uncertainties
    elif idx:
        uncertainties_dict[idx] = np.array([0])
print('sort_probs:\n', sort_probs)
print('uncertainties_dict:\n', uncertainties_dict)
# deepAL+: return unlabeled_idxs[uncertainties.sort()[1][:n]] 
sorted_uncertainties_list = sorted(uncertainties_dict.items(), key=lambda x: x[1], reverse=True)
    
unlabeled_idxs[[idx for (idx, uncertainties) in sorted_uncertainties_list[:5]]]

sort_probs:
 [0.21894264 0.06242085 0.05455256 0.04888514 0.04601725 0.04060458
 0.03590286 0.03363937 0.03167988 0.02968265 0.02858026 0.02661794
 0.02554646 0.02404774 0.02287048 0.01757931 0.01394101 0.01358491
 0.01289108 0.01246493 0.01230124 0.01199799 0.01062952 0.01053182
 0.00970575 0.00929304 0.00913635 0.00905683 0.00811593 0.00736513
 0.00716131 0.00668042 0.00546745 0.00521209 0.0048835  0.00473575
 0.00424123 0.00387308 0.00370081 0.00363776 0.00342435 0.00328293
 0.00309034 0.00269852 0.00267169 0.00254447 0.00250732 0.00250326
 0.00225909 0.00212089 0.00205793 0.00202384 0.00182608 0.00182338
 0.00175312 0.00174881 0.00161135 0.00160185 0.00152893 0.00141906
 0.00139053 0.00123604 0.00103741 0.00093622 0.00084983 0.00072486
 0.00060394 0.00054503]
uncertainties_dict:
 {0: 0.019561194, 1: 0.0061373785, 2: 0.003160715, 3: 0.045883723, 4: 0.028362542, 5: 0.04024148, 6: 0.016859218, 7: 0.0030643567, 8: 0.046888124, 9: 0.008154124, 10: 0.042255454, 11: 0.02228225, 12: 0.0035

array([19, 18, 13, 14,  8])

In [ ]:
# import DataLoader
# import default_data_collator
import itertools
def margin_sampling_query(n_pool, labeled_idxs, train_dataset, train_features, examples, model, device, n):
    # deepAL+: unlabeled_idxs, unlabeled_data = self.dataset.get_unlabeled_data()
    unlabeled_idxs, unlabeled_data = get_unlabel_data(n_pool, labeled_idxs, train_dataset)
    unlabeled_features = train_features.select(unlabeled_idxs)
    
    unlabeled_dataloader = DataLoader(
		unlabeled_data,
		shuffle=True,
		collate_fn=default_data_collator,
		batch_size=8,
	)

    # deepAL+: probs = self.predict_prob(unlabeled_data)
    prob_dict = get_prob(model, unlabeled_dataloader, device, unlabeled_features, examples)
    
    # deepAL+: probs_sorted, _ = probs.sort(descending=True)
    # deepAL+: uncertainties = probs_sorted[:, 0] - probs_sorted[:,1]
    uncertainties_dict = {}
    for idx, probs in prob_dict.items():
        if len(probs) > 1: # if prob_dict['probs'] is not 0
            sort_probs = np.sort(probs)[::-1] # This method returns a copy of the array, leaving the original array unchanged.
            uncertainties = sort_probs[0] - sort_probs[1]
            uncertainties_dict[idx] = uncertainties
        elif idx:
            uncertainties_dict[idx] = np.array([0])
    print('sort_probs:\n', sort_probs)
    # deepAL+: return unlabeled_idxs[uncertainties.sort()[1][:n]] 
    sorted_uncertainties_dict = sorted(uncertainties_dict.items(), key=lambda x: x[1], reverse=True)
    sorted_uncertainties_dict = dict(itertools.islice(sorted_uncertainties_dict.items(), n))
    
    return unlabeled_idxs[list(sorted_uncertainties_dict.keys())]

In [ ]:
def get_prob_dropout(model, eval_dataloader, device, features, examples, n_drop=10):
    model.train()
    
    prob_dict = {}
    for_check = []
    
    for i in range(n_drop):
        
        start_logits = []
        end_logits = []
        # accelerator.print("Evaluation!")
        for batch in tqdm(eval_dataloader):
            batch = {key: value.to(device) for key, value in batch.items()}
            with torch.no_grad():
                outputs = model(**batch)

            start_logits.append(outputs.start_logits.cpu().numpy())
            end_logits.append(outputs.end_logits.cpu().numpy())

        start_logits = np.concatenate(start_logits)
        end_logits = np.concatenate(end_logits)
        start_logits = start_logits[: len(val_dataset)]
        end_logits = end_logits[: len(val_dataset)]

        example_to_features = collections.defaultdict(list)
        max_answer_length = 30
        n_best = 20
            
        for idx, feature in enumerate(features):
            example_to_features[feature["example_id"]].append(idx)

        n = 0
        for example in tqdm(examples):
            example_id = example["id"]
            # context = example["context"]
            answers = []

            # Loop through all features associated with that example
            for feature_index in example_to_features[example_id]:
                start_logit = start_logits[feature_index]
                end_logit = end_logits[feature_index]
                offsets = features[feature_index]["offset_mapping"]

                start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
                end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
                for start_index in start_indexes:
                    for end_index in end_indexes:
                        # Skip answers that are not fully in the context
                        if offsets[start_index] is None or offsets[end_index] is None:
                            continue
                        # Skip answers with a length that is either < 0 or > max_answer_length
                        if (
                            end_index < start_index
                            or end_index - start_index + 1 > max_answer_length
                        ):
                            continue

                        answers.append(start_logit[start_index] + end_logit[end_index])

            
            if 1 < len(answers) < 150:
                zero_list = [0] * (150 - len(answers))
                answers.extend(zero_list)
            else:
                answers[:150]

            if len(answers) > 1:
                if example_to_features[example_id][0] not in prob_dict:
                    prob_dict[example_to_features[example_id][0]] = softmax(answers)
                else:
                    prob_dict[example_to_features[example_id][0]] += softmax(answers)
            elif example_to_features[example_id] != []:
                if example_to_features[example_id][0] not in prob_dict:
                    prob_dict[example_to_features[example_id][0]] = np.array([0])
            # if n == 0 and len(softmax(answers)) > 1:
            #     for_check.append(answers[:5])
            #     n += 1      

    for key in prob_dict.keys():
        prob_dict[key] /= n_drop
    # return prob_dict, for_check
    return prob_dict
# move to evaluation.py

In [ ]:
probs_list_dict_20_dropout = get_prob_dropout(model, unlabeled_dataloader, device, unlabeled_data_20, squad['train'])

In [ ]:
for_check
# the prediction are the same

In [ ]:
probs_list_dict_20_dropout

In [ ]:
probs_list_dict_20

In [ ]:
confidence_list_dict = []
for d in probs_list_dict_20:
    if len(d['probs']) > 1: # if prob_dict['probs'] is not 0
        confidence = max(d['probs'])
        confidence_list_dict.append(
            {'idx': d['idx'], 
                'confidence': confidence}
                )
    elif d['idx']:
        confidence_list_dict.append(
            {'idx': d['idx'], 
                'confidence': np.array([0])}
                )
# deepAL+: return unlabeled_idxs[uncertainties.sort()[1][:n]]
sorted_confidence_dict = sorted(confidence_list_dict, key=lambda d: d['confidence'])   
unlabeled_idxs_20[[confidence_dict['idx'][0] for confidence_dict in sorted_confidence_dict[:5]]]

In [ ]:
def get_entropy(prob_list_dict, NUM_QUERY):
    entropy_list_dict = []
    for d in prob_list_dict:
        if len(d['probs']) > 1: # if prob_dict['probs'] is not 0
            log_probs = np.log(d['probs'])
            entropy_list_dict.append(
                {'idx': d['idx'], 
                'entropy': (d['probs']*log_probs).sum()}
                )
        elif d['idx']:
            entropy_list_dict.append(
                {'idx': d['idx'], 
                'entropy': np.array([0])}
                )
    # deepAL+: return unlabeled_idxs[uncertainties.sort()[1][:n]] # use smallest Entropy
    sorted_entropy_dict = sorted(entropy_list_dict, key=lambda d: d['entropy'], reverse=True) # use largest Entropy, different from deepAL+
    return unlabeled_idxs[[entropy_dict['idx'][0] for entropy_dict in sorted_entropy_dict[:NUM_QUERY]]]

In [ ]:
q_idxs_20 = get_entropy(probs_list_dict_20, 5)
q_idxs_20

In [ ]:
preds, _, _ = trainer_qs.predict(unlabeled_data_20)

In [ ]:
start_logits, end_logits = preds
start_logits

In [ ]:
# predict with unlable data
preds, _, _ = trainer_qs.predict(unlabeled_data_20)

In [ ]:
start_logits, end_logits = preds
start_logits

In [ ]:
q_idxs = get_entropy(probs_list_dict, NUM_QUERY)
q_idxs